[healpix]: https://healpix.jpl.nasa.gov
[swift]: http://swift.asdc.asi.it/
[swift_deepsky]: https://github.com/chbrandt/swift_deepsky


# Sky discretization with HEALpix

The task we want to accomplish is that of covering a given region of the sky with circles. Not a new problem to the human existence, I want to do this exercise using [Healpix][healpix].

Healpix will be used here for binning a series of coordinates; the coordinates may be a real list of sources or an artifical one.
In any case, the binning will downsample the list of coordinates.

An application for the downsampling is the definition of observational pointings on a survey, where we have to define the minimum amount of to cover a specific region.

The pointings are defined by a central coordinate -- RA,Dec -- and a radius value defining the size of the circle (i.e, field of view) each pointing covers.


## Swift/XRT

At the end of the day we want to define the coverage of a region as if done by the [Swift/XRT][swift] telescope.
Such list of pointings will then be used by our [Swift-DeepSky][swift_deepsky] pipeline.

The Swift' field-of-view is of $12\ arcmin$.

In [1]:
from astropy import units

bbox = dict(ramin=21 * units.degree,
            ramax=23 * units.degree,
            decmin=32 * units.degree,
            decmax=34 * units.degree)

rad = 12 *units.arcmin

In [2]:
# get the closest smaller size of healpix elements

from moc import utils
level = utils.size_to_level(rad, truncate=True)

from moc import core
dsize = core.HEALPIX_LEVELS[level]

print("Healpix' level {:d} corresponds to a size '{}'".format(level,dsize))

Healpix' level 8 corresponds to a size '13.74 arcmin'


In [3]:
# create a grid of (fake) coordinates, to then create a MOC table from it
step_size = dsize.to('deg').value/2**0.5

import numpy as np
ra_vec = np.arange(bbox['ramin'].value, bbox['ramax'].value, step_size)
dec_vec = np.arange(bbox['decmin'].value, bbox['decmax'].value, step_size)

In [4]:
import itertools
grid = list(itertools.product(ra_vec,dec_vec))

In [5]:
from astropy.coordinates import SkyCoord
coords = SkyCoord(grid,unit='deg')

## A MOC catalog (with hierarchical coverage)

In [6]:
from mocpy import MOC
moc = MOC.from_coo_list(coords, level)

In [7]:
tmpfile = 'moc.json'
moc.write(tmpfile, format='json')

import json
with open(tmpfile, 'r') as fp:
    dmoc = json.load(fp)
    
import os
os.remove(tmpfile)

In [8]:
from healpy import pixelfunc

def moc_to_pointings(moc_dict):
    coords_dict = {}
    for k,v in moc_dict.items():
        level = int(k)
        nside = 2**level
        coords = []
        for ipix in v:
            c = pixelfunc.pix2ang(nside, ipix, nest=True, lonlat=True)
            coords.append(c)
        coords = list(zip(*coords))
        size = core.HEALPIX_LEVELS[level]
        coords_dict[level] = dict(radius=size, ra=coords[0], dec=coords[1])
    return coords_dict

### Plot

In [9]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

Loading BokehJS ...

In [10]:
coords_moc = moc_to_pointings(dmoc)

colors = ['green','yellow','blue','red']
levels = list(map(int,coords_moc.keys()))

fig = figure()
for i in range(len(levels)):
    lvl = levels[i]
    clr = colors[i]

    x = coords_moc[lvl]['ra']
    y = coords_moc[lvl]['dec']
    r = coords_moc[lvl]['radius'].to('deg').value

    fig.circle(x, y, radius=r, fill_alpha=0.25, color=clr)
    
show(fig)

## A list of fixed-order Healpix elements

In [11]:
# Read the coordinates representing (center) each element of our interest

from healpy import pixelfunc

# defined earlier:
# -ra_vec
# -dec_vec
# -radius
# -level

nside = 2**level

pix_to_visit = set()
for ra,dec in grid:
    ipix = pixelfunc.ang2pix(nside, ra, dec, nest=True, lonlat=True)
    pix_to_visit.add(ipix)

coords_to_visit = []
for ipix in pix_to_visit:
    c = pixelfunc.pix2ang(nside, ipix, nest=True, lonlat=True)
    coords_to_visit.append(c)

    ## We don't need the neighbours, but if that was the case:
    #
    #nipixs = pixelfunc.get_all_neighbours(nside, c[0], c[1], nest=True, lonlat=True)
    #cns = []
    #for nipix in nipixs:
    #    cns.append(pixelfunc.pix2ang(nside, nipix, nest=True, lonlat=True))
    #coords_to_visit.extend(cns)

In [12]:
x_hp,y_hp = list(zip(*coords_to_visit))
r_hp = core.HEALPIX_LEVELS[level].to('deg').value
r_og = rad.to('deg').value

print('radii:', r_hp, r_og)

radii: 0.229 0.2


In [13]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

fig = figure()
fig.circle(x_hp,y_hp,radius=r_og, fill_alpha=0.1, color='red')
fig.circle(x_hp,y_hp,radius=r_hp, fill_alpha=0.1)
show(fig)

Loading BokehJS ...

## Create the list of pointings

In [14]:
import pandas

df = pandas.DataFrame({'ra':x_hp, 'dec':y_hp, 'radius':r_hp*60})
df

,dec,ra,radius
0,31.914005,21.093750,13.74
1,31.914005,21.445312,13.74
2,32.089951,21.621094,13.74
3,32.089951,21.269531,13.74
4,32.266237,21.445312,13.74
5,32.442866,22.675781,13.74
6,32.089951,20.917969,13.74
7,32.266237,21.093750,13.74
8,32.442866,21.269531,13.74
9,32.442866,20.917969,13.74


In [15]:
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
def sds_cmdline(df_row):
    cmdline = 'docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky'
    ra = df_row['ra']
    dec = df_row['dec']
    radius = df_row['radius']
    lbl = '{:d}_{:d}_{:d}'.format(int(df_row['index']), int(ra), int(dec))
    cmdline += ' --ra {:.5f} --dec {:.5f} --radius {:.3f} --label {!s}'
    cmdline = cmdline.format(ra, dec, radius, lbl)
#     print(cmdline)
    return cmdline
    
cmdlines = df.reset_index().apply(sds_cmdline, axis=1)

In [17]:
cmdlines.to_csv('swift_deepsky_pointings_list.txt', header=False, index=False)

In [18]:
%cat swift_deepsky_pointings_list.txt

docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 23.02734 --dec 33.51006 --radius 13.740 --label 0_23_33
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 22.67578 --dec 33.86870 --radius 13.740 --label 1_22_33
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 21.26953 --dec 32.79717 --radius 13.740 --label 2_21_32
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 22.14844 --dec 33.68919 --radius 13.740 --label 3_22_33
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 22.50000 --dec 33.68919 --radius 13.740 --label 4_22_33
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 21.97266 --dec 33.51006 --radius 13.740 --label 5_21_33
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 21.26953 --dec 32.08995 --radius 13.740 --label 6_21_32
docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky --ra 22.85156 --dec 34.04859 --radius 13.740 --label 7_22_34
docker run --rm 